# Demo of Fitting Property Correlations

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from fluids import (
    AntoineCorrelation,
    ExponentialCorrelation,
    PolynomialCorrelation,
)

In [ ]:
DATA_DIR = "data/properties/fluids"
FILENAME = "SYLTHERM800_data.csv"


def load_syltherm_data():
    """Load SYLTHERM 800 manufacturer data fixture"""
    df = pd.read_csv(os.path.join(DATA_DIR, FILENAME))

    data = {
        "T_K": df["Temperature_C"].values + 273.15,
        "T_C": df["Temperature_C"].values,
        "density": df["Density_kg_m3"].values,
        "heat_capacity": df["Heat_Capacity_kJ_kg_K"].values
        * 1000,  # Convert to J/kg·K
        "thermal_conductivity": df["Thermal_Conductivity_W_m_K"].values,
        "viscosity": df["Viscosity_mPa_s"].values
        / 1000,  # Convert to Pa·s
        "vapor_pressure": df["Vapor_Pressure_kPa"].values
        * 1000,  # Convert to Pa
    }

    return data

syltherm_data = load_syltherm_data()
syltherm_data = pd.DataFrame(syltherm_data)
syltherm_data

In [ ]:
# Choose temperature range for fitting
T_min_C = 200  # degC
T_max_C = 400  # degC

# Filter data to selected temperature range
T_C = syltherm_data["T_C"]
selected_data = syltherm_data.loc[(T_C >= T_min_C) & (T_C <= T_max_C)]
selected_data.describe()

## Fit polynomial correlation to density data

In [ ]:
density_fit_params = {
    "bounds": {
        "a0": (800, 1400),  # Intercept
        "a1": (-2, 2),  # Linear term
        "a2": (-0.01, 0.01),  # Quadratic term
    },
    "initial_guess": {"a0": 1200, "a1": -0.5, "a2": 0},
}

T_data = selected_data["T_K"]
y_data = selected_data["density"]

# Create a quadratic correlation model (2nd order polynomial)
poly_corr = PolynomialCorrelation(
    coefficients=[0, 0, 0],  # Will be fitted
    T_min=T_min_C + 273.15,
    T_max=T_max_C + 273.15,
    name="Density Fitted",
)

# Fit to data
fit_results = poly_corr.fit(
    T_data,
    y_data,
    bounds=density_fit_params["bounds"],
    initial_guess=density_fit_params["initial_guess"],
)
fit_results

In [ ]:
T_values = np.linspace(T_data.min(), T_data.max(), 100)
fig, axes = poly_corr.plot_correlation(T_data=T_data, y_data=y_data)
plt.tight_layout()
plt.show()

## Fit exponential correlation to viscosity data

Andrade equation - exponential decrease with temperature

In [ ]:
viscosity_fit_params = {
    "bounds": {
        "A": (1e-8, 1e-3),  # Pre-exponential factor
        "B": (500, 5000),  # Activation energy parameter
        "C": (-1e-3, 1e-3),  # Offset
    },
    "initial_guess": {"A": 1e-6, "B": 2000, "C": 0},
}

T_data = selected_data["T_K"]
y_data = selected_data["viscosity"]

# Create a linear correlation model (2nd order polynomial)
exp_corr = ExponentialCorrelation(
    A=1e-6,
    B=2000,
    C=0,  # Will be fitted
    T_min=T_min_C + 273.15,
    T_max=T_max_C + 273.15,
    name="Viscosity Fitted",
)

# Fit to data
fit_results = exp_corr.fit(
    T_data,
    y_data,
    bounds=viscosity_fit_params["bounds"],
    initial_guess=viscosity_fit_params["initial_guess"],
)
fit_results

In [ ]:
T_values = np.linspace(T_data.min(), T_data.max(), 100)
fig, axes = exp_corr.plot_correlation(T_data=T_data, y_data=y_data)
plt.tight_layout()
plt.show()

## Fit linear correlation to heat capacity

In [ ]:
heat_capacity_fit_params = {
    "bounds": {
        "a0": (500, 2000),  # Intercept
        "a1": (0, 5),  # Slope term
    },
    "initial_guess": {"a0": 1000.0, "a1": 1.5},
}

T_data = selected_data["T_K"]
y_data = selected_data["heat_capacity"]

# Create a linear correlation model (2nd order polynomial)
poly_corr = PolynomialCorrelation(
    coefficients=[0, 0],  # Will be fitted
    T_min=T_min_C + 273.15,
    T_max=T_max_C + 273.15,
    name="Heat Capacity Fitted",
)

# Fit to data
fit_results = poly_corr.fit(
    T_data,
    y_data,
    bounds=heat_capacity_fit_params["bounds"],
    initial_guess=heat_capacity_fit_params["initial_guess"],
)
fit_results

In [ ]:
T_values = np.linspace(T_data.min(), T_data.max(), 100)
fig, axes = poly_corr.plot_correlation(T_data=T_data, y_data=y_data)
plt.tight_layout()
plt.show()

## Fit Antoine correlation to vapour pressure

In [ ]:
vapor_pressure_fit_params = {
    "bounds": {
        "A": (0, 30),  # Antoine constant
        "B": (500, 2000),  # Antoine constant
        "C": (-10, 200),  # Antoine constant
    },
    "initial_guess": {"A": 10.0, "B": 2300, "C": -10.0},
}

T_data = selected_data["T_K"]
y_data = selected_data["vapor_pressure"]

# Create Antoine correlation
antoine_corr = AntoineCorrelation(
    A=10,
    B=1000,
    C=3,  # Will be fitted
    T_min=T_min_C + 273.15,
    T_max=T_max_C + 273.15,
    name="Vapor Pressure Fitted",
)

# There is a problem with the data for vapor pressure at low temperatures.
# The vapor pressure is stated as zero due to rounding errors.  Therefore,
# discard data for temperatures where vapor pressure data is zero.

# Fit to data
fit_results = antoine_corr.fit(
    T_data[y_data > 0],
    y_data[y_data > 0],
    bounds=vapor_pressure_fit_params["bounds"],
    initial_guess=vapor_pressure_fit_params["initial_guess"],
)
fit_results

In [ ]:
T_values = np.linspace(T_data.min(), T_data.max(), 100)
fig, axes = antoine_corr.plot_correlation(T_data=T_data, y_data=y_data)
plt.tight_layout()
plt.savefig("vapor_pressure_fit.png", dpi=300)
plt.show()

## Fit linear correlation to thermal conductivity

In [ ]:
thermal_conductivity_fit_params = {
    "bounds": {
        "a0": (0, 10),  # Intercept
        "a1": (-2, 0),  # Slope term
    },
    "initial_guess": {"a0": 1000.0, "a1": 1.5},
}

T_data = selected_data["T_K"]
y_data = selected_data["thermal_conductivity"]

# Create a linear correlation model (2nd order polynomial)
poly_corr = PolynomialCorrelation(
    coefficients=[0, 0],  # Will be fitted
    T_min=T_min_C + 273.15,
    T_max=T_max_C + 273.15,
    name="Thermal Conductivity Fitted",
)

# Fit to data
fit_results = poly_corr.fit(
    T_data,
    y_data,
    bounds=thermal_conductivity_fit_params["bounds"],
    initial_guess=thermal_conductivity_fit_params["initial_guess"],
)
fit_results

In [ ]:
T_values = np.linspace(T_data.min(), T_data.max(), 100)
fig, axes = poly_corr.plot_correlation(T_data=T_data, y_data=y_data)
plt.tight_layout()
plt.show()